In [77]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from modeler_strats.universal_modeler import UniversalModeler
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks
from transformer.analysis_transforms import Analysis as anas

import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b
import math

In [78]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [79]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
current_db = ADatabase("btc_current")
umod = UniversalModeler()

In [80]:
market.connect()
prices = market.retrieve("crypto")
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [81]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [54]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 1
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [82]:
market.connect()
prices = market.retrieve("crypto")
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [84]:
speculation_db.connect()
simulation = speculation_db.retrieve("predictions")
speculation_db.disconnect()

In [85]:
classification_db.connect()
classification_simulation = classification_db.retrieve("predictions")
classification_db.disconnect()

In [86]:
ticker_sim = prices.copy()
price_returns = risks.weekly_risk_prep(ticker_sim,bench_returns)

In [87]:
sim = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1,errors="ignore")[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [88]:
current_db.connect()
current_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = risks.weekly_backtest_prep(strat,backtest_data)
    b.weeklies_backtest(backtest_data.copy(),positions,parameter,start_date,end_date,current_db)
current_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:01<00:00, 39.62it/s]
